In [1]:
#Importing basic packages
import os
import warnings
import requests
import numpy as np
import pandas as pd
import pandas as pd
import calendar
import datetime

#Visualisations Libraries
import matplotlib.pyplot as plt
import plotly.express as px 
import squarify 
import seaborn as sns 
from pprint import pprint as pp 
from plotly.subplots import make_subplots 
import plotly.graph_objects as go
from Levenshtein import distance

### Data Standardization and Modeling with K-Means and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from yellowbrick.cluster import SilhouetteVisualizer

In [2]:
# !/usr/bin/env/ python
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import urllib
import pyodbc

# import tqdm as tqdm
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import numpy as n
import os
import json
from datetime import date  

In [3]:
# SQL and snow flake connection
os.chdir("C:\\Users\\prapa001\\OneDrive - Corporate\\Desktop\\python_trials") 

credentials= json.load(open("credentials.json"))

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=WINMPNDBp02;"
            "Database=ANALYSIS_PROJECTS;"
            "UID="+credentials['SQL']['user'] + ";" 
            +"pwd=" + credentials['SQL']['password'] +";" +
            "Trusted_Connection=Yes;"
           ) 
sql_connection = pyodbc.connect(cnxn_str)

sf_connection = snowflake.connector.connect( 
    user =credentials['SF']['user'], 
    password=credentials['SF']['password'] ,
    role='SF_SCM_ANALYTICS_DBRL',
    account='staples.east-us-2.azure', 
    warehouse='CAP_PRD_SC_WH',
    database='DATALAB_SANDBOX',
    schema='SCM_ANALYTICS',
    authenticator='externalbrowser' 
    ) 

engine = create_engine(URL(
    user =credentials['SF']['user'], 
    password=credentials['SF']['password'],
    role='SF_SCM_ANALYTICS_DBRL',
    account='staples.east-us-2.azure', 
    warehouse='CAP_PRD_SC_WH',
    database='DATALAB_SANDBOX',
    schema='SCM_ANALYTICS',
    authenticator='externalbrowser' 
)) 

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
Active_SKU_FC = '''SELECT FC_Number, 
       FC_Name, 
       COUNT(DISTINCT Staples_SKU) AS Count_of_SKUs,
       COUNT(DISTINCT CASE WHEN PURCHASE_STATUS = 'A' THEN STAPLES_SKU ELSE '0' END) AS Active_Staples_SKU  
FROM LINKED.SCCDATA.FC_MASTER_V    
GROUP BY FC_Number, FC_Name'''
 # SKU Level DF
Active_SKU_FC_01 =  pd.read_sql(Active_SKU_FC,sql_connection)

C:\Users\prapa001\AppData\Local\Temp\1\ipykernel_33572\3013709443.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Active_SKU_FC_01 =  pd.read_sql(Active_SKU_FC,sql_connection)


In [13]:
# Sorting Active SKUs by descending order
Active_SKU_FC_01 = Active_SKU_FC_01.sort_values(by = 'Active_Staples_SKU', ascending = False).head(20)
# Create the bar chart
fig = px.bar(Active_SKU_FC_01, x='FC_Name', y='Active_Staples_SKU', text='Active_Staples_SKU',text_auto='.3s',color='FC_Name')
# Add labels inside the bars
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor='rgba(0, 0, 0, 0)')

# Set the title of the chart
fig.update_layout(title_text='No of Active SKUs at FC level')
# Set the label color to white

# Show the chart
fig.show()

In [17]:
# P1 to P3 Pick Type level cartons and count of SKU's
Pick_Type = '''SELECT  PICK_TYPE, 
        COUNT(DISTINCT CTN_ID) AS Count_of_Cartons,
      --  COUNT(Distinct CASE WHEN pick_type = 'CONVEY BREAKPACK' THEN CTN_ID END) As Total_BP_Cartons, 
      --  COUNT(Distinct CASE WHEN pick_type = 'FULL CASE' THEN  CTN_ID END) As Total_FC_Cartons, 
        COUNT(DISTINCT SHPD_SKU) AS Count_of_SKUs        
FROM [COST_TO_SERVE_ARCHIVE].[SC_Cost].[Carton_Pick_List_SC_Costs_Archive]  
WHERE     STAT_IND <> '99' 
      AND PICK_CTL_CHAR NOT IN ('#','T') 
      AND PICK_TYPE NOT IN ('DUMMY WRAP AND LABEL', 'RSI', 'DNR') 
      AND [Unit_Zero_Valid] IS NULL 
      AND [Valid_Period_Record] = 'Y' 
      AND  YEAR IN  ('2023')
      AND BU = 'CONTRACT'
      AND TimePeriod IN ('1_CTS','2_CTS','3_CTS')
      AND FC = '00974'       
GROUP BY Pick_TYPE'''
 # SKU Level DF
Pick_Type_01 =  pd.read_sql(Pick_Type,sql_connection)

C:\Users\prapa001\AppData\Local\Temp\ipykernel_20776\3523196016.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [18]:
Pick_Type_01 = Pick_Type_01.style.format({
'Count_of_Cartons': '{:,.0f}',
'Count_of_SKUs': '{:,.0f}'})
# P1 - P3 Pick Type Level Aggregation Summary 
Pick_Type_01 

,PICK_TYPE,Count_of_Cartons,Count_of_SKUs
0,CONVEY BREAKPACK,"199,940","12,336"
1,FULL CASE,"379,286","1,922"
2,NON CONVEY BREAKPACK,"68,453","1,041"
3,PICKS FROM RESERVE,"28,154","1,654"
4,WRAP AND LABEL,"20,880",2


In [22]:
FC_SKUs = '''SELECT  shpd_sku, 
          COUNT(DISTINCT CTN_ID) AS Count_of_Cartons
      --  COUNT(DISTINCT CASE WHEN pick_type = 'CONVEY BREAKPACK' THEN CTN_ID END) As Total_BP_Cartons
FROM [COST_TO_SERVE_ARCHIVE].[SC_Cost].[Carton_Pick_List_SC_Costs_Archive]  
WHERE     STAT_IND <> '99' 
      AND PICK_CTL_CHAR NOT IN ('#','T') 
      AND PICK_TYPE NOT IN ('DUMMY WRAP AND LABEL', 'RSI', 'DNR') 
      AND [Unit_Zero_Valid] IS NULL 
      AND [Valid_Period_Record] = 'Y' 
      AND  YEAR IN  ('2023')
      AND TimePeriod = '4_CTS'
      AND BU = 'CONTRACT'
      AND FC = '00974'
      AND pick_type = 'FULL CASE'
GROUP BY shpd_sku'''
 # SKU Level DF
FC_SKUs_01 =  pd.read_sql(FC_SKUs,sql_connection)

C:\Users\prapa001\AppData\Local\Temp\ipykernel_20776\3569293954.py:17: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [40]:
Master_query = '''SELECT A.shpd_sku,
       A.Count_of_Cartons,
       B.SKU_Dept,
       B.SKU_Class,
       B.SKU_Dept_Name,
       B.SKU_Class_Name, 
       B.SKU_Description,
       B.FC_DIMs_Length,
       B.FC_DIMs_Width,
       B.FC_DIMs_Height,
       B.FC_DIMs_Weight_lbs,
       B.FC_DIMs_Volume,
       B.SKU_Dept,
       C.SKU, 
       C.FC, 
       C.Seasonal_4_WK_FCST
FROM (SELECT shpd_sku, 
             COUNT(DISTINCT CTN_ID) AS Count_of_Cartons
      -- COUNT(DISTINCT CASE WHEN pick_type = 'CONVEY BREAKPACK' THEN CTN_ID END) AS Total_BP_Cartons
      FROM [COST_TO_SERVE_ARCHIVE].[SC_Cost].[Carton_Pick_List_SC_Costs_Archive]  
      WHERE STAT_IND <> '99' 
        AND PICK_CTL_CHAR NOT IN ('#','T') 
        AND PICK_TYPE NOT IN ('DUMMY WRAP AND LABEL', 'RSI', 'DNR') 
        AND [Unit_Zero_Valid] IS NULL 
        AND [Valid_Period_Record] = 'Y' 
        AND YEAR IN ('2023')
        AND BU = 'CONTRACT'
        AND TimePeriod IN ('4_CTS')
        AND FC = '00974'
        AND pick_type = 'FULL CASE'
      GROUP BY shpd_sku) A 
LEFT JOIN (
  SELECT DISTINCT SKU_NUM,
         SKU_Dept,
         SKU_Class,
         SKU_Dept_Name,
         SKU_Class_Name, 
         SKU_Description,
         FC_DIMs_Length,
         FC_DIMs_Width,
         FC_DIMs_Height,
         FC_DIMs_Weight_lbs,
         FC_DIMs_Volume
  FROM LINKED.PRISM.MASTER_DETAIL_CURRENT_V
) B ON A.shpd_sku = B.SKU_NUM 
LEFT JOIN (
  SELECT * 
  FROM OPENQUERY(SNWFLK_SC_ANALYTICS_FA,
    'SELECT SKU, 
            FC, 
            Seasonal_4_WK_FCST 
     FROM DATALAB_SANDBOX.SCM_REPLEN.NADTRIM
     WHERE FC = ''974'' AND RUN_DATE = ''2023-05-02'''
  ) C ON A.shpd_sku = C.SKU '''
 # SKU Level DF 
Master_query_01 =  pd.read_sql(Master_query,sql_connection)

IndentationError: unexpected indent (466133701.py, line 54)

In [ ]:
Master_file =pd.read_excel('C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\SC Strategic Projects\\LA - MIRADA CONS\\5123_SDOProject\\Redo Analysis\\Master_file.xlsx')
Master_file['2-Week-Forecast'] = Master_file['Seasonal_4_WK_FCST']/2 
Master_file

In [ ]:
# calculate total usage units
total_cartons = Master_file['Count of Cartons'].sum()
# Usage Percentage 
Master_file['usage_percentage'] = (Master_file['Count of Cartons'] / total_cartons) *100
# Total Cartons and Cumulative SUM
Master_file = Master_file.sort_values(by = 'Count of Cartons',ascending = False)  
Master_file['Cumulative_sum'] = Master_file["Count of Cartons"].cumsum()
Master_file['cum_perc'] = round(100*Master_file.Cumulative_sum/Master_file["Count of Cartons"].sum(),2)
# Considering 85 % Scenario 
# Assuming your cumulative sum column is named 'cumulative_sum'
Per_85_Scenario = Master_file[Master_file['Cumulative_sum'] <= Master_file['Cumulative_sum'].max() * 0.85]
# Assuming your dataframe is named 'df' and the column you want to rank is 'column_name'
Per_85_Scenario['rank'] = Per_85_Scenario['Count of Cartons'].rank(ascending=False)
# Multiple per unit volume cubic to 2 WOS 
Per_85_Scenario['Total_Space'] = Per_85_Scenario['FC_DIMs_Volume'] * Per_85_Scenario['2-Week-Forecast']

In [25]:
# Data Prepation for 85 Percent scenario or 233 SKUs 
Master_File_01 = Per_85_Scenario[["SHPD SKU","Total_Space","2-Week-Forecast"]]
Master_File_01.set_index('SHPD SKU', inplace=True)
Master_File_01.head()
# Standard Scaling for the 233 sku and make shpd sku

# Define StandardScaler object
scaler = StandardScaler()

# Fit the StandardScaler object to the Master_File_01 DataFrame
Master_file_02_std_copy = scaler.fit_transform(Master_File_01)

# Create a DataFrame from the standardized data
Master_file_05_std_copy = pd.DataFrame(Master_file_02_std_copy, columns=Master_File_01.columns, index=Master_File_01.index)

# Reset the index of the DataFrame
Master_file_05_std_copy = Master_file_05_std_copy.reset_index()

# Replacing NAN with Mean 
Master_file_05_std_copy[['Total_Space','2-Week-Forecast']] = Master_file_05_std_copy[['Total_Space','2-Week-Forecast']].fillna(value = Master_file_05_std_copy[['Total_Space','2-Week-Forecast']].mean())
# Performing K-means within 6 Clusters as it 6 clusters was threshold where we saw maximum decline
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42)

# Fitting our model to the dataset
kmeans.fit(Master_file_05_std_copy)

# Adding Cluster labels
Master_file_05_std_copy["Cluster"] = kmeans.labels_

,Total_Space,2-Week-Forecast
SHPD SKU,,
135855,2.251887e+07,108500.000
24483894,9.201905e+06,49208.050
24377968,7.552180e+06,6449.695
24377969,1.356292e+06,1948.275
571863,2.826838e+06,2114.235


In [54]:
# CLustering model for all the 1922 Full Case SKUs
# Total Space calculation 
Master_file['Total_Space'] = Master_file['FC_DIMs_Volume'] * Master_file['2-Week-Forecast']

# Data Prepation for 85 Percent scenario or 233 SKUs 
Master_File_03 = Master_file[["SHPD SKU","Total_Space","2-Week-Forecast"]]
Master_File_03.set_index('SHPD SKU', inplace=True)
Master_File_03.head()
# Standard Scaling for the 1922 sku and make shpd sku

# Define StandardScaler object
scaler = StandardScaler()

# Fit the StandardScaler object to the Master_File_01 DataFrame
MF_07_std_copy = scaler.fit_transform(Master_File_03)

# Create a DataFrame from the standardized data
MF_06_std_copy = pd.DataFrame(MF_07_std_copy, columns=Master_File_03.columns, index=Master_File_03.index)

# Reset the index of the DataFrame
MF_06_std_copy = MF_06_std_copy.reset_index()

# Replacing NAN with Mean 
MF_06_std_copy[['Total_Space','2-Week-Forecast']] = MF_06_std_copy[['Total_Space','2-Week-Forecast']].fillna(value = MF_06_std_copy[['Total_Space','2-Week-Forecast']].mean())
# Performing K-means within 6 Clusters as it 6 clusters was threshold where we saw maximum decline
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42)

# Fitting our model to the dataset
kmeans.fit(MF_06_std_copy)

# Adding Cluster labels
MF_06_std_copy["Cluster"] = kmeans.labels_


C:\Users\prapa001\Anaconda3\envs\customer_analytics\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [58]:
MF_06_std_copy

Master_file_05_std_copy
# Join the original df to scaled df to tie up the cluster associated with each SKU
MF_2k_SKUs = Master_file.merge(MF_06_std_copy, on='SHPD SKU', how='left', indicator=True)
# Move this file to excel 
# Transfer the SKU level file to excel
with pd.ExcelWriter('C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\SC Strategic Projects\\LA - MIRADA CONS\\5123_SDOProject\\Stra_cluster_2K_SKUs.xlsx') as writer : 
    MF_2k_SKUs.to_excel(writer, index=False)

In [40]:
Master_file_05_std_copy
# Join the original df to scaled df to tie up the cluster associated with each SKU
Master_file_05 = Master_File_01.merge(Master_file_05_std_copy, on='SHPD SKU', how='left', indicator=True)
Master_file_05

,SHPD SKU,Total_Space_x,2-Week-Forecast_x,Total_Space_y,2-Week-Forecast_y,Cluster,_merge
0,135855,2.251887e+07,108500.000,12.594502,13.441896,5,both
1,24483894,9.201905e+06,49208.050,4.949082,6.006224,0,both
2,24377968,7.552180e+06,6449.695,4.001956,0.643993,0,both
3,24377969,1.356292e+06,1948.275,0.444827,0.079479,0,both
4,571863,2.826838e+06,2114.235,1.289084,0.100292,1,both
...,...,...,...,...,...,...,...
228,665223,5.915520e+04,39.500,-0.299874,-0.159896,1,both
229,2831689,5.216087e+04,36.255,-0.303889,-0.160303,4,both
230,934100,3.236659e+05,46.280,-0.148015,-0.159046,3,both
231,24398921,9.409321e+05,83.315,0.206365,-0.154401,0,both


In [47]:
# Count the number of SKUs at cluster level
Cluster_groupby = Master_file_05.groupby(['Cluster']).agg({
    'SHPD SKU': ['count'],
    'Total_Space_x': ['sum'],
    '2-Week-Forecast_x': ['sum']
}).reset_index()

# Reset the column index
Cluster_groupby.columns = Cluster_groupby.columns.get_level_values(0) + '_' + Cluster_groupby.columns.get_level_values(1)

# Calculate Usage % and Volume %
SKU_Usage_sum = Cluster_groupby['2-Week-Forecast_x_sum'].sum()
SKU_Volume = Cluster_groupby['Total_Space_x_sum'].sum()
Cluster_groupby['Usage %'] = (Cluster_groupby['2-Week-Forecast_x_sum'] / SKU_Usage_sum) * 100
Cluster_groupby['Volume %'] = (Cluster_groupby['Total_Space_x_sum'] / SKU_Volume) * 100

# Format the DataFrame
Cluster_groupby_formatted = Cluster_groupby.copy()

# Apply desired formatting to the columns
Cluster_groupby_formatted['Usage %'] = Cluster_groupby_formatted['Usage %'].map('{:.2f}%'.format)
Cluster_groupby_formatted['Volume %'] = Cluster_groupby_formatted['Volume %'].map('{:.2f}%'.format)

# Print the formatted DataFrame
Cluster_groupby_formatted

,Cluster_,SHPD SKU_count,Total_Space_x_sum,2-Week-Forecast_x_sum,Usage %,Volume %
0,0,34,2.648062e+07,69015.405,22.73%,19.71%
1,1,77,4.763448e+07,68789.760,22.65%,35.46%
2,2,9,2.538825e+06,1028.080,0.34%,1.89%
3,3,76,2.077429e+07,28323.705,9.33%,15.47%
4,4,15,3.194153e+06,1931.045,0.64%,2.38%
5,5,22,3.369991e+07,134563.080,44.32%,25.09%


In [45]:
# Transfer the SKU level file to excel
with pd.ExcelWriter('C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\SC Strategic Projects\\LA - MIRADA CONS\\5123_SDOProject\\Stra_cluster_method_85_sample_8_Clusters.xlsx') as writer : 
    Master_file_05.to_excel(writer, index=False)

In [46]:
Cluster_groupby 

,Cluster_,SHPD SKU_count,Total_Space_x_sum,2-Week-Forecast_x_sum
0,0,34,2.648062e+07,69015.405
1,1,77,4.763448e+07,68789.760
2,2,9,2.538825e+06,1028.080
3,3,76,2.077429e+07,28323.705
4,4,15,3.194153e+06,1931.045
5,5,22,3.369991e+07,134563.080
